In [23]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import matplotlib.colors as colors

In [28]:
# Helper functions

def rgb_to_hsv(patches):
    new_patches = [colors.rgb_to_hsv(patches[i]) for i in range(len(patches))]
    return np.asarray(new_patches)

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

In [29]:
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

n = 10 # Only use 10 images for training

Loading 20 images
satImage_001.png
Loading 20 images
satImage_001.png


In [30]:
# Extract patches from input images
patch_size = 4 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [31]:
# Extract 6-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(feat_m, feat_v)
    return feat

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
    #X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
    return X

In [32]:
# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

Training classifier

In [33]:
from sklearn import linear_model
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Perform feature scaling
print('Scaling features:')
X_scaler = StandardScaler().fit(X)
scaled_X = X_scaler.transform(X)

# Train linear SVC classifier
print('Training classifier:')
svc = SVC()
svc.fit(scaled_X,Y)
accuracy = round(svc.score(scaled_X, Y), 4)

print('Accuracy =', accuracy)


Scaling features...
    ...Done
Training classifier...
    ...Done
Accuracy = 0.7859


Helpers

In [34]:
# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*255

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

def bw(gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = gt_img #np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        cimg = gt_img_3c
    return cimg
    

Predictions

In [35]:
import mask_to_submission
from PIL import Image
from scipy.misc import imsave

print ("Running prediction on test set")
prediction_training_dir = "test_set_images/"#"predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, 50+1):  
    #Reading the test images
    path = prediction_training_dir +'test_'+ str(i)
    ims = os.listdir(path)
    #Scaling the images
    Xi = extract_img_features(path+'/'+ims[0])
    X_scaler = StandardScaler().fit(Xi)
    scaled_X = X_scaler.transform(Xi)
    #Prediction
    Zi = svc.predict(scaled_X) 
    im = load_image(path+'/'+ims[0])
    w = im.shape[0]
    h = im.shape[1]
    predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
    pimg = Image.fromarray(bw(predicted_im))
    pimg.save(prediction_training_dir + "pred_" + str(i) + ".png") 





Running prediction on test set


/Users/inesbahej/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


KeyboardInterrupt: 

In [22]:
from mask_to_submission import *


submission_filename = 'submission.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = 'test_set_images/pred_' + str(i) + '.png'
    print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)



test_set_images/pred_1.png
test_set_images/pred_2.png
test_set_images/pred_3.png
test_set_images/pred_4.png
test_set_images/pred_5.png
test_set_images/pred_6.png
test_set_images/pred_7.png
test_set_images/pred_8.png
test_set_images/pred_9.png
test_set_images/pred_10.png
test_set_images/pred_11.png
test_set_images/pred_12.png
test_set_images/pred_13.png
test_set_images/pred_14.png
test_set_images/pred_15.png
test_set_images/pred_16.png
test_set_images/pred_17.png
test_set_images/pred_18.png
test_set_images/pred_19.png
test_set_images/pred_20.png
test_set_images/pred_21.png
test_set_images/pred_22.png
test_set_images/pred_23.png
test_set_images/pred_24.png
test_set_images/pred_25.png
test_set_images/pred_26.png
test_set_images/pred_27.png
test_set_images/pred_28.png
test_set_images/pred_29.png
test_set_images/pred_30.png
test_set_images/pred_31.png
test_set_images/pred_32.png
test_set_images/pred_33.png
test_set_images/pred_34.png
test_set_images/pred_35.png
test_set_images/pred_36.png
t